# Classifying dog species with Convolutional Neural Networks
DAT300, group CA2-10. Jon Nordby and Espen Sønneland.


## Kaggle setup
Downloads data from https://www.kaggle.com/c/dat300-2018-dogs

In [1]:
!pip install -q kaggle && echo Kaggle installed

Kaggle installed


In [2]:
from google.colab import drive
import shutil, os, os.path
drive.mount('/content/drive')
if not os.path.exists('/root/.kaggle'):
  os.makedirs('/root/.kaggle/')
shutil.copyfile('/content/drive/My Drive/kaggle.json', '/root/.kaggle/kaggle.json')
'Kaggle API key installed'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


'Kaggle API key installed'

In [3]:
!kaggle competitions download -c dat300-2018-dogs -p data/
!unzip -n -q 'data/*.zip' -d data/
!ls data/train/*.jpg | wc -l
!ls data/test/*.jpg | wc -l

  0% 0.00/109k [00:00<?, ?B/s]
100% 109k/109k [00:00<00:00, 47.4MB/s]
  0% 0.00/202k [00:00<?, ?B/s]
100% 202k/202k [00:00<00:00, 74.8MB/s]
 97% 361M/371M [00:05<00:00, 69.7MB/s]
100% 371M/371M [00:05<00:00, 71.0MB/s]
 94% 346M/369M [00:05<00:00, 54.6MB/s]
100% 369M/369M [00:06<00:00, 64.3MB/s]

3 archives were successfully processed.
10290
10290


In [4]:
import time
import os
import re

import pandas
import numpy
import matplotlib.pyplot as plt

import sklearn

import sklearn.preprocessing
import sklearn.model_selection
import sklearn.pipeline

import sklearn.linear_model
import sklearn.ensemble
import sklearn.svm

import keras
import keras.wrappers.scikit_learn

Using TensorFlow backend.


In [5]:
labels = pandas.read_csv('data/labels.csv', index_col='id')

print(labels.shape)
labels.head(3)

(10290, 1)


,breed
id,
0,soft-coated_wheaten_terrier
1,Tibetan_terrier
2,Lhasa


In [6]:
print('number of breeds', len(labels.breed.unique()))
print('images per breed (average)', len(labels)/len(labels.breed.unique()))
 

number of breeds 120
images per breed (average) 85.75


Under 100 images per class is considered **very few** for training deep learning models.

In [7]:
# Create a pandas.DataFrame with the samples (id, filename)
def files_dataframe(directory):
  import glob
  import re
  files = {}
  for filename in glob.iglob(directory+'/*.jpg'):
    m = re.findall('dog(\d+).jpg', filename)
    file_id = int(m[0])
    files[file_id] = os.path.basename(filename)
  filenames = list(files.values())

  ids = list(files.keys())
  df = pandas.DataFrame({
      'filename': filenames,
      'filepath': [ os.path.join(directory, f) for f in filenames ],
  }, index=ids)
  df.sort_index(inplace=True) # make sure ordered by id
  assert df.filename.values[0] == 'dog0.jpg'
  return df

train_files = files_dataframe('data/train')
_ = files_dataframe('data/test') # just ensure it runs without error
train_data = train_files.join(labels)
train_data.head(3)

,filename,filepath,breed
0,dog0.jpg,data/train/dog0.jpg,soft-coated_wheaten_terrier
1,dog1.jpg,data/train/dog1.jpg,Tibetan_terrier
2,dog2.jpg,data/train/dog2.jpg,Lhasa


## Training CNN from scratch



In [0]:
def build_model(image_size=(150, 150), n_classes=120, complexity=2, dropout=0.3):
  from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
  from keras.layers import Dropout, Activation, Dense, Flatten
  from keras.layers.normalization import BatchNormalization

  model = keras.Sequential()

  in_shape = (image_size[0], image_size[1], 3)
  model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=in_shape))
  model.add(BatchNormalization(axis=3, scale=False))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
  model.add(Dropout(dropout))

  if complexity > 1:
    model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3, scale=False))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
    model.add(Dropout(dropout))

  if complexity > 2:
    model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3, scale=False))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
    model.add(Dropout(dropout))

  if complexity > 3:
    model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
    model.add(BatchNormalization(axis=3, scale=False))
    model.add(Activation("relu"))
    model.add(Dropout(dropout))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dense(n_classes, activation='softmax'))
  
  model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy', 'top_k_categorical_accuracy'])
  return model

In [0]:
def train_model(train_dir, complexity=3):
  image_size = (120, 120)
  epochs = 20
  batch_size = 32
  labels = pandas.read_csv('data/labels.csv', index_col='id')
  labeled_data = files_dataframe(train_dir).join(labels)
  
  # Setup model
  model = build_model(image_size=image_size,complexity=complexity, dropout=0.2)
  model.summary()
  
  outdir = '/content/drive/My Drive/dat300-dogs'
  weigthspath = os.path.join(outdir, 'scratch.c{}.best.hdf5'.format(complexity))
  from keras.callbacks import ModelCheckpoint
  checkpointer = ModelCheckpoint(filepath=weigthspath, 
                                 verbose=1, save_best_only=True)
  

  # Setup data augmentation
  from keras.preprocessing.image import ImageDataGenerator
  train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    vertical_flip=False,
    horizontal_flip=True)
  
  test_datagen = ImageDataGenerator(rescale=1./255)

  # Split training and validation data
  train_data = labeled_data.iloc[0:7000]
  val_data = labeled_data.iloc[7000:]
  # flow_from_dataframe raises Exception if we dont start from 0...
  val_data['i'] = range(0, len(val_data))
  val_data.set_index('i', inplace=True)
  
  # Train model
  train_generator = train_datagen.flow_from_dataframe(
          train_data,
          train_dir,
          y_col='breed',
          target_size=image_size,
          batch_size=batch_size,
          class_mode='categorical')

  validation_generator = test_datagen.flow_from_dataframe(
          val_data,
          train_dir,
          y_col='breed',
          target_size=image_size,
          batch_size=batch_size,
          class_mode='categorical')
  
  model.fit_generator(
        train_generator,
        steps_per_epoch=4000//batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=2000//batch_size,
        callbacks=[checkpointer], verbose=1)


fromscratch = train_model('data/train')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_73 (Conv2D)           (None, 120, 120, 16)      432       
_________________________________________________________________
batch_normalization_72 (Batc (None, 120, 120, 16)      48        
_________________________________________________________________
activation_72 (Activation)   (None, 120, 120, 16)      0         
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 30, 30, 32)        4608      
_________________________________________________________________
batch_normalization_73 (Batc (None, 30, 30, 32)        96        
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Found 7000 images belonging to 120 classes.
Found 3290 images belonging to 120 classes.
Epoch 1/20
 38/125 [========>.....................] - ETA: 27s - loss: 5.1380 - acc: 0.0066 - top_k_categorical_accuracy: 0.0436

Tried many combinations of `complexity` (1-4), `dropout` (0.2-0.5), and `batch_size` (20-200) and image_sizes (120,224) - but unable to make the network train well. Usually ends up overfitting within 3-5 epochs, at losses of `~4.80` - which is practically the equal probability performance.

This is suprising, since the architecture of this CNN was able to get 11% accuracy on a 133 class dog breed problem, without use of data augmentation. [machinememos.com: dog-breed-image-classifcation](http://machinememos.com/python/keras/artificial%20intelligence/machine%20learning/transfer%20learning/dog%20breed/neural%20networks/convolutional%20neural%20network/tensorflow/image%20classification/imagenet/2017/07/11/dog-breed-image-classification.html)